In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train,y_train),(x_test,y_test)= fashion_mnist.load_data()


In [4]:
x_train, y_train = x_train[6000:], y_train[6000:]
x_valid, y_valid = x_train[:6000], y_train[:6000]
len_train = len(x_train)
len_valid = len(x_valid)
print(len_train,len_valid)

54000 6000


In [5]:
x_train = x_train.reshape(-1,28,28,1)
x_valid = x_valid.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)

In [6]:

y_train[:10]

array([8, 6, 4, 4, 6, 5, 5, 8, 7, 3], dtype=uint8)

In [10]:
train_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_generator.fit(x_train)
train_gen = train_generator.flow(x_train,y_train,
                     shuffle=True,
                    )

test_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_generator.fit(x_test)
test_gen = test_generator.flow(x_test,y_test)

valid_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
valid_generator.fit(x_valid)
valid_gen = valid_generator.flow(x_valid,y_train)
                    

In [7]:
model = keras.models.Sequential([
    keras.layers.Conv2D(32, 3, padding='same', input_shape=(28,28,1), activation = 'relu'),
    keras.layers.Conv2D(32, 3,padding='same', activation = 'relu'),
    keras.layers.MaxPool2D(),
    keras.layers.Conv2D(64, 3, padding='same',  activation = 'relu'),
    keras.layers.Conv2D(64, 3,padding='same', activation = 'relu'),
    keras.layers.MaxPool2D(),
#     keras.layers.Conv2D(128, 3, padding='same', activation = 'relu'),
#     keras.layers.Conv2D(128, 3,padding='same', activation = 'relu'),
#     keras.layers.MaxPool2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10)
    ])
model.compile(loss =keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                     optimizer = 'adam',
                     metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0

In [15]:
callbacks = [keras.callbacks.EarlyStopping(patience=5,min_delta=0.01,)]
history = model.fit(train_gen, epochs = 5, steps_per_epoch=54000//32,validation_data = valid_gen,validation_steps = 6000//32)

Train for 1687 steps
Epoch 1/5
 123/1687 [=>............................] - ETA: 8:54 - loss: 0.9366

KeyboardInterrupt: 

In [ ]:
model.evaluate(test_gen, verbose=2)

In [ ]:
pd.DataFrame(history.history).plot()